<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/notebooks/2021_0502kmnist_fashionMNIST_etc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 畳込みニューラルネットワークによる日本語古典籍くずし字 kminst の認識実習

<div align='right'>
<a href='mailto:2019cnps@gmail.com'>2019cnps@gmai.com</a>, all rights reserved.<br>
Date: 31/Aug/2019<br>
 MIT license
</div>

---

- kmnist とは日本語のくずし字データセットです。
[論文](https://arxiv.org/abs/1812.01718) や [プロジェクトページ](http://codh.rois.ac.jp/kmnist/)
の提供している画像認識のためのデータセットです。

| File            | Examples | Download (MNIST format)    | Download (NumPy format)      |
|-----------------|--------------------|----------------------------|------------------------------|
| Training images | 60,000             | [train-images-idx3-ubyte.gz](http://codh.rois.ac.jp/kmnist/dataset/kmnist/train-images-idx3-ubyte.gz) (18MB) | [kmnist-train-imgs.npz](http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-train-imgs.npz) (18MB)   |
| Training labels | 60,000             | [train-labels-idx1-ubyte.gz](http://codh.rois.ac.jp/kmnist/dataset/kmnist/train-labels-idx1-ubyte.gz) (30KB) | [kmnist-train-labels.npz](http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-train-labels.npz) (30KB)  |
| Testing images  | 10,000             | [t10k-images-idx3-ubyte.gz](http://codh.rois.ac.jp/kmnist/dataset/kmnist/t10k-images-idx3-ubyte.gz) (3MB) | [kmnist-test-imgs.npz](http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-test-imgs.npz) (3MB)   |
| Testing labels  | 10,000             | [t10k-labels-idx1-ubyte.gz](http://codh.rois.ac.jp/kmnist/dataset/kmnist/t10k-labels-idx1-ubyte.gz) (5KB)  | [kmnist-test-labels.npz](http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-test-labels.npz) (5KB) |

実際のデータを以下に示します

![](http://codh.rois.ac.jp/img/kmnist.png)

- 一枚の画像が縦横 28 画素。訓練画像が 60,000枚，テスト画像が 10,000 枚あります。


## kmnist データセットのダウンロード

In [ ]:
# 早速ダウンロードしてみましょう
import requests

kmnist_urllist = ['http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-train-imgs.npz',
                'http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-train-labels.npz',
                'http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-test-imgs.npz',
                'http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-test-labels.npz']

for url in kmnist_urllist:
  path = url.split('/')[-1]
  r = requests.get(url)
  with open(path, 'wb') as f:
    total_length = int(r.headers.get('content-length'))
    print('Downloading {0} - {1} bytes'.format(path, (total_length)))
    f.write(r.content)

In [ ]:
# 4 つの .npz ファイルが存在すればダウンロード成功です
!ls -l *.npz

In [ ]:
# ダウロードしてきたファイルを numpy のデータとして読み込みます
import numpy as np

# Load the data
x_test  = np.load('kmnist-test-imgs.npz')['arr_0']
y_test  = np.load('kmnist-test-labels.npz')['arr_0']
x_train = np.load('kmnist-train-imgs.npz')['arr_0']
y_train = np.load('kmnist-train-labels.npz')['arr_0']

In [ ]:
# データサイズを確認してみましょう
for data in [x_test, y_test, x_train, y_train]:
  print(data.shape)

## データの確認

In [ ]:
# kminst は 10 文字を識別する問題であるため人間にわかりやすいように正解データと対応する
# 文字のデータベースを作っておきます

labels = 'おきすつなはまやれを'
labels_dict = {i:c for i, c in enumerate(labels)}
print(labels_dict)

In [ ]:
!pip install japanize_matplotlib
import japanize_matplotlib

In [ ]:
y_stats = np.zeros((10),dtype=np.int)
for y in y_train:
    y_stats[y]+=1
print(y_stats)

### 任意のデータを表示してみましょう

下のセルの最終行の `n=数字` の部分に好みの数字を入れてデータを表示してみましょう

何度か数字を入れ替えて試してみましょう

In [ ]:
import matplotlib.pyplot as plt

def show(n=0, data=x_train):
  if n > len(data):
    raise Exception('n は 0 から {0} 以下の整数でなければなりません {1}'.format(n, len(data)))
  x = x_train[n]
  label = y_train[n]
  plt.figure(figsize=(1.2, 1.2))
  plt.axis('off')
  plt.title('正解:{0}'.format(labels_dict[label]))
  plt.imshow(x,cmap='gray')
  plt.show()

target = 9
max_count = 5
count = 0; i = 0
while count <= max_count:
    if y_train[i] == target:
        show(i)
        count += 1
    i += 1        


In [ ]:
# 複数のデータを一度に表示してみましょう。下の数字を入れ替えて試してみてください。
# それぞれの意味は
# start: データを表示する最初の番号
# w: 一行あたりの文字数
# h: 一列あたりの文字数
start = 20000
start = 0
w = 10
h = 2
idx = 0
canvas = np.zeros((28 * h, 28 * w))
for i in range(h):
    for j in range(w):
        canvas[i*28:(i+1)*28, j*28:(j+1)*28] = x_train[start + idx].reshape((28, 28))
        idx += 1
plt.figure(figsize=(w, h))
plt.axis('off')
plt.imshow(canvas, cmap='gray')
plt.show()

for i in range(h):
  for j in range(w):
    print(labels_dict[y_train[start + j + i * w]], end=' ')
  print()

In [ ]:
# 表示精度桁数の設定
np.set_printoptions(suppress=False, formatter={'float': '{:6.3f}'.format})

X = x_train.reshape(-1,28*28)[:1000]
inv_XtX = np.linalg.inv(X.T.dot(X))
inv_XtX_Xt = np.dot(inv_XtX, X.T)

y = np.zeros((1000,10))
y[np.arange(1000),y_train[:1000]] = 1

w = np.dot(inv_XtX_Xt, y)

#print(w.shape)
y_hat = np.dot(X[:1000],w)
print(np.argmax(y_hat[:20],axis=1))
#print(y[:10])
print(y_train[:20])


In [ ]:
A = [i == j for i, j in zip(y_hat, y_train)]
print(np.sum(A))

In [ ]:
from statsmodels import regression
help(regression.linear_model)


In [ ]:
import numpy as np
import statsmodels.api as sm
data = sm.datasets.longley.load(as_pandas=False)
data.exog = sm.add_constant(data.exog)
#data.keys()
#help(data['data'])
#help(data.endog)
#help(data.exog)
#help(sm.OLS)
ols_resid = sm.OLS(y[:1000], X[:1000]).fit()
help(sm.OLS(y,X).fit())
#help(ols_resid) #.params)
#help(ols_resid)
#ols_resid = sm.OLS(data.endog, data.exog).fit().resid
#res_fit = sm.OLS(ols_resid[1:], ols_resid[:-1]).fit()
#rho = res_fit.params

## 簡単な2層のニューラルネットワークを作ってみましょう

### まず keras などを輸入 `import` して準備します

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import random

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Conv2D, MaxPooling2D, Flatten

from keras.layers import Activation

In [ ]:
model = Sequential()
model.add(Dense(100, activation='sigmoid', input_dim=784)) # = 28 x 28 
model.add(Dense(100, activation='sigmoid'))
model.add(Dense(10, activation='softmax'))
model.summary()

In [ ]:
num_classes = 10

# kmnist データを変形します。
# 訓練データは 60000行 784列 である必要があります
# テストデータは 10000行 784列です
x_train = x_train.reshape(60000, 28*28)
x_test = x_test.reshape(10000, 28*28)

# 64 ビットにはしません。なぜなら GPU での計算は 32 ビットに限定されるからです
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# データの最大値で割って 0 から 1 までにしておきます
x_train /= 255
x_test /= 255

# keras で扱えるように正解ラベルをクラスベクトルに変換します
# この場合 ワンホットベクトルにすることを意味します
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print('%d train samples, %d test samples'%(x_train.shape[0], x_test.shape[0]))
print("training data shape: ", x_train.shape, y_train.shape)
print("test data shape: ", x_test.shape, y_test.shape)

In [ ]:
# 損失関数 `loss`, 最適化手法 `optimizer`, 損失関数の計算手法を指定して
# 分析の設定を行います
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [ ]:
# 実際の訓練には入力データ，教師データ，ミニバッチのサイズ，エポック数，評価データを指定して
# 訓練を実施してみましょう
model.fit(x_train, y_train,
          batch_size=128,
          epochs=30,
          validation_data=(x_test, y_test))

## 訓練結果を表示してみます

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# 層を深くしてみます

In [ ]:
from keras import backend as K

# input image dimensions
img_rows, img_cols = 28, 28

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [ ]:
#x_train = x_train.reshape(60000, 28*28)
#x_test = x_test.reshape(10000, 28*28)
print(x_train.shape)

In [ ]:
model = Sequential()
model.add(Conv2D(64, (3, 3), padding='same', input_shape=(28,28,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.summary()

In [ ]:
print('%d train samples, %d test samples'%(x_train.shape[0], x_test.shape[0]))
print("training data shape: ", x_train.shape, y_train.shape)
print("test data shape: ", x_test.shape, y_test.shape)

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=30,
          validation_data=(x_test, y_test))

## 結果を表示してみます

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])